# Cell Data Preparation

In this notebook the raw data contained in the [OpenCell ID dataset](https://opencellid.org/#zoom=16&lat=41.14945&lon=-8.61079) is cleaned and prepared for later use. The dataset contains the position and range of mobile antennas spread around the world.

### Find map BBox

Given the coordinates (north west corner) and zoom level defined to get a [map of the centre of Milan](https://www.openstreetmap.org/#map=14/45.4646/9.1721), find the coordinates of the south east corner to get a boundary box (BBox) for the map tile. Reference [here](https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Python).

In [25]:
import math

# geo:45.4646,9.1721?z=14
nw_lat_degree = 45.4646
nw_lon_degree = 9.1721
zoom = 14

# find tile number
lat_rad = math.radians(nw_lat_degree)
n = 1 << zoom
x_tile_num = int((nw_lon_degree + 180.0) / 360.0 * n)
y_tile_num = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)

# find nw corner of the next tile
se_lon_degree = (x_tile_num + 1) / n * 360.0 - 180.0
lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * (y_tile_num + 1) / n)))
se_lat_degree = math.degrees(lat_rad)

BBox = (
    nw_lon_degree, # min longitude
    se_lon_degree, # max longitude
    se_lat_degree, # min latitude
    nw_lat_degree, # max latitude
)

print(BBox)
with open("data/BBox.txt", "w") as f:
    print(BBox, file=f)

(9.1721, 9.1845703125, 45.46013063792099, 45.4646)


### Import of the dataset

In [3]:
import pandas as pd

In [42]:
italy_cells_1 = pd.read_csv(
    "assets/italy_cells_1.csv",
    names=["radio", 'mcc', 'mnc', 'area', 'cell', "unit", 'lon', 'lat', 'range', "samples", "changeable", "created", "updated", "avg_signal"],
    usecols=["mcc", "mnc", "area", "cell", "lon", "lat"],
)

italy_cells_2 = pd.read_csv(
    "assets/italy_cells_2.csv",
    names=["radio", 'mcc', 'mnc', 'area', 'cell', "unit", 'lon', 'lat', 'range', "samples", "changeable", "created", "updated", "avg_signal"],
    usecols=["mcc", "mnc", "area", "cell", "lon", "lat"],
)

cell_dataset = pd.concat([italy_cells_1, italy_cells_2])

In [44]:
cell_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3686474 entries, 0 to 1439027
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   mcc     int64  
 1   mnc     int64  
 2   area    int64  
 3   cell    int64  
 4   lon     float64
 5   lat     float64
dtypes: float64(2), int64(4)
memory usage: 196.9 MB


In [43]:
cell_dataset.head()

,mcc,mnc,area,cell,lon,lat
0,208,10,18013,8653445,2.272110,48.980484
1,208,10,16505,47046446,2.540156,50.485447
2,208,20,80,17939,-1.229193,49.292793
3,208,20,80,18060,-1.214537,49.290525
4,208,1,29191,26486747,7.172216,43.753281


### Duplicates elimination

Each cell is uniquely identified by the tuple `(MCC, MNC, Area, Cell)`. Since the dataset contain some minor data duplication (more than one row for the same cell), here duplicates are removed.


In [45]:
original_row_number = len(cell_dataset)
cleaned_dataset = cell_dataset.drop_duplicates(
    subset=['mcc', 'mnc', 'area', 'cell'],
    inplace=False
)
print(f"{original_row_number - len(cleaned_dataset)} duplicates have been deleted")

12284 duplicates have been deleted


### Drop not used columns

In [48]:
cleaned_dataset.drop(columns=['mcc', 'mnc', 'area'], inplace=True)

/var/folders/5d/xx8nncdn3f3fh2s7gf6xv7vr0000gq/T/ipykernel_2819/565520308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_dataset.drop(columns=['mcc', 'mnc', 'area'], inplace=True)


### Save

In [49]:
cleaned_dataset.to_parquet("assets/italy_cell_cleaned.parquet", index=False)

### Filter towers in Milan and save

In [50]:
milan_cells = cleaned_dataset[
    (cleaned_dataset.lon >= BBox[0]) &
    (cleaned_dataset.lon <= BBox[1]) &
    (cleaned_dataset.lat >= BBox[2]) &
    (cleaned_dataset.lat <= BBox[3])
]

In [51]:
milan_cells.reset_index(drop=True, inplace=True)

In [54]:
milan_cells.to_parquet("assets/milan_cells.parquet", index=False)

### Porto cells

In [1]:
LON_MIN = -8.6338
LON_MAX = -8.5862
LAT_MIN = 41.1369
LAT_MAX = 41.1690
BBox = (LON_MIN, LON_MAX, LAT_MIN, LAT_MAX)

In [4]:
porto_cells = pd.read_csv(
    "assets/portugal_cells.csv",
    names=["radio", 'mcc', 'mnc', 'area', 'cell', "unit", 'lon', 'lat', 'range', "samples", "changeable", "created", "updated", "avg_signal"],
    usecols=["mcc", "mnc", "area", "cell", "lon", "lat"],
)

In [5]:
original_row_number = len(porto_cells)
cleaned_dataset = porto_cells.drop_duplicates(
    subset=['mcc', 'mnc', 'area', 'cell'],
    inplace=False
)
print(f"{original_row_number - len(cleaned_dataset)} duplicates have been deleted")

625 duplicates have been deleted


In [6]:
cleaned_dataset.drop(columns=['mcc', 'mnc', 'area'], inplace=True)

/var/folders/5d/xx8nncdn3f3fh2s7gf6xv7vr0000gq/T/ipykernel_10211/565520308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_dataset.drop(columns=['mcc', 'mnc', 'area'], inplace=True)


In [7]:
porto_cells = cleaned_dataset[
    (cleaned_dataset.lon >= BBox[0]) &
    (cleaned_dataset.lon <= BBox[1]) &
    (cleaned_dataset.lat >= BBox[2]) &
    (cleaned_dataset.lat <= BBox[3])
]

In [8]:
porto_cells.reset_index(drop=True, inplace=True)

In [9]:
porto_cells.to_parquet("assets/porto_cells.parquet", index=False)